In [35]:
%pip install transformers
%pip install torch
%pip install tokenizers
%pip install nltk
%pip install sentence-transformers
%pip install parascore==1.0.5
%pip install spacy
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_sm

%pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
/Users/chelseashe/Thought_Toolkit/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 6.3 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
/Users/chelseashe/Thought_Toolkit/.venv/lib/python3.9/site-packag

In [36]:
import utils

import json
import os
import re
from tqdm import tqdm

from parascore import ParaScorer
import Levenshtein

import spacy
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
from argminer.data import PersuadeProcessor

import level_2_learning_comparisons
import action_parser
from extract_coauthor_raw_logs import jsonl_names
import outcome_measures
import level_3_comparisons

from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
import re

### Set Up

### Setup

In [37]:
# #Extracting logs from zip file
# import zipfile

# folder_name = '/content/vibewritingpilot.zip'

# with zipfile.ZipFile(folder_name, 'r') as zip_ref:
#     zip_ref.extractall('/content/pilot2')  # This will extract to /content/pilot_2

In [38]:
def load_jsonl_files_parsed(folder_path):
    """
    Reads all .jsonl files in the given folder and returns a dictionary.

    Each key in the dictionary is the file name (without extension),
    and its value is a list of parsed JSON objects (not strings).
    """
    jsonl_data = {}

    for filename in os.listdir(folder_path):
        if filename.endswith('.jsonl'):
            full_path = os.path.join(folder_path, filename)
            try:
                with open(full_path, 'r', encoding='utf-8') as file:
                    lines = file.read().splitlines()
                    parsed_lines = [json.loads(line) for line in lines]
                    jsonl_data[filename.split('.')[0]] = parsed_lines
            except Exception as e:
                print(f"Error reading {filename}: {e}")

    return jsonl_data

# Example usage
folder_path = "pilot_raw_logs"  # Your folder path
data = load_jsonl_files_parsed(folder_path)

# Print a preview of one file to confirm the format
for file_name, events in data.items():
    print(f"\nFile: {file_name}, Number of events: {len(events)}")
    if events:
        print("First event:")
        print(json.dumps(events[0], indent=2))  # Pretty-print first event
    break  # Remove this if you want to preview more files


File: adee2b5ddc4f4ead92bd719a202d6f7c, Number of events: 1947
First event:
{
  "eventName": "system-initialize",
  "eventSource": "api",
  "eventTimestamp": 1751278758472,
  "textDelta": "",
  "cursorRange": "",
  "currentDoc": "\n",
  "currentCursor": 0,
  "currentSuggestions": [],
  "currentSuggestionIndex": 0,
  "currentHoverIndex": "",
  "currentN": "",
  "currentMaxToken": "",
  "currentTemperature": "",
  "currentTopP": "",
  "currentPresencePenalty": "",
  "currentFrequencyPenalty": "",
  "originalSuggestions": []
}


In [39]:
jsonl_data = {}
filename = "pilot_raw_logs/f9da5086528a4eeab85bbf1b0f8bb279.jsonl"
# Print a preview of one file to confirm the format
with open(filename, 'r', encoding='utf-8') as file:
    lines = file.read().splitlines()
    parsed_lines = [json.loads(line) for line in lines]
    jsonl_data[filename.split('.')[0]] = parsed_lines

data = jsonl_data
print(data)

{'pilot_raw_logs/f9da5086528a4eeab85bbf1b0f8bb279': [{'eventName': 'system-initialize', 'eventSource': 'api', 'eventTimestamp': 1750169276554, 'textDelta': '', 'cursorRange': '', 'currentDoc': '\n', 'currentCursor': 0, 'currentSuggestions': [], 'currentSuggestionIndex': 0, 'currentHoverIndex': '', 'currentN': '', 'currentMaxToken': '', 'currentTemperature': '', 'currentTopP': '', 'currentPresencePenalty': '', 'currentFrequencyPenalty': '', 'originalSuggestions': []}, {'eventName': 'text-insert', 'eventSource': 'user', 'eventTimestamp': 1750170154661, 'textDelta': {'ops': [{'insert': 'W'}]}, 'cursorRange': '', 'currentDoc': '', 'currentCursor': 1, 'currentSuggestions': [], 'currentSuggestionIndex': 0, 'currentHoverIndex': '', 'currentN': '5', 'currentMaxToken': '50', 'currentTemperature': '0.95', 'currentTopP': '1', 'currentPresencePenalty': '0.5', 'currentFrequencyPenalty': '0.5', 'originalSuggestions': []}, {'eventName': 'text-insert', 'eventSource': 'user', 'eventTimestamp': 17501701

In [40]:
logs_by_session = data

In [41]:
for key in logs_by_session.keys():
  logs_by_session[key] = logs_by_session[key][1:]

### Loading Structured JSON File for Parsing and Analysis

Once the `extract_coauthor_raw_logs.py` script has successfully generated a structured JSON file, you can proceed to load the file in this section for further parsing and analysis.

In [42]:
# Use the current working directory
script_dir = os.getcwd()

# Get the raw log dataset
file_path = os.path.join(script_dir, 'pilot3_logs.json')

# Feel free to uncomment the line below and start with a smaller sample (20 writing sessions) to reduce runtime
# file_path = os.path.join(script_dir, 'small_logs_for_test.json')

# Open and load the JSON file
with open(file_path) as f:
    logs_by_session = json.load(f)

In [43]:
"""
# Load raw JSON-like data from file
file_path = os.path.join(script_dir, '/content/testing_new_site.json')
with open(file_path, 'r') as f:
    raw_data = f.read()
"""

raw_data = logs_by_session


def fix_all_arrays(raw_json_string):
    def fix_array(match):
        array_content = match.group(1)
        # Add commas between objects: }{
        fixed = re.sub(r'\}\s*\{', '},\n{', array_content.strip())
        return f'[{fixed}]'

    # Regex: match any array content following a key
    fixed_json_string = re.sub(r'\[\s*({.*?})\s*\]', lambda m: fix_array(m), raw_json_string, flags=re.DOTALL)
    return fixed_json_string


# This regex looks for a pattern where a closing brace is immediately followed (with any whitespace) by an opening brace,
# and inserts a comma between them.
raw_data_str = json.dumps(logs_by_session)
fixed_data = re.sub(r'(\})\s*(\{)', r'\1, \2', raw_data_str)


# Now try to load the fixed data as JSON.
try:
    logs_by_session = json.loads(fixed_data)
    print("✅ Fixed and loaded JSON successfully!")
except json.JSONDecodeError as e:
    print("❌ Failed to parse JSON:", e)


for test_user, events in logs_by_session.items():
    if isinstance(events, list):
       logs_by_session[test_user] = [
            event for event in events if event.get("eventName") != "system-initialize"
        ]

✅ Fixed and loaded JSON successfully!


In [44]:
def are_strings_similar_lev(str1, str2, max_differences=4):
    distance = Levenshtein.distance(str1, str2)
    return distance <= max_differences

### Parsing Raw Log JSON File into Structured Level 1 Actions

This section processes raw logs and converts them into Level 1 actions using a analyzer. Each parsed action is enriched with a **level_1_action_type** key, which specifies the action type (e.g., `insert_text`, `delete_text`, `accept_suggestion`).

**level_1_actions_per_session** is a dictionary where each session key maps to a list of parsed actions, organizing the output by session for streamlined analysis and further processing.

In [45]:
# Use the current working directory
script_dir = os.getcwd()

In [46]:
ai_sentences = []


def split_insert_text_by_delta(action, prev_action, threshold=5):
    """
    Splits an 'insert_text' action into AI and human based on insert length from delta.
    """
    try:
        ops = action["action_logs"][0]["textDelta"]["ops"]
        inserted_text = ops[1].get("insert", "")
    except (IndexError, KeyError, TypeError):
        action["level_1_action_type"] = "insert_text_human"
        action["human_sentences_temporal_order"] = " ".join(
            action["sentences_temporal_order"]
        )
        return [action]

    if prev_action["level_1_action_type"] == "present_suggestion" and len(inserted_text.strip()) >= threshold:
        # ---- AI action ----
        ai_action = action.copy()
        ai_action["action_logs"] = ai_action["action_logs"][0]
        ai_action["action_delta"] = [
            "INSERT",
            inserted_text,
            action["action_delta"][2],
            action["action_delta"][3],
        ]
        ai_action["action_modified_sentences"] = utils.sent_tokenize(inserted_text)
        ai_sentences.extend(ai_action["action_modified_sentences"])
        ai_action["action_end_writing"] = (
            ai_action["action_start_writing"] + inserted_text
        )
        ai_action["level_1_action_type"] = "insert_text_ai"

        # ---- Human action ----
        action_human = action.copy()
        action_human["action_start_writing"] = ai_action["action_end_writing"]
        remaining_text = action_human["action_end_writing"][
            len(action_human["action_start_writing"]) :
        ]
        action_human["action_delta"] = [
            "INSERT",
            remaining_text,
            action["action_delta"][2],
            action["action_delta"][3],
        ]
        action_human["action_logs"] = action_human["action_logs"][1:]
        action_human["level_1_action_type"] = "insert_text_human"

        # ---- Filter modified sentences ----
        action_human["action_modified_sentences"] = [
            s
            for s in action_human["action_modified_sentences"]
            if not any(
                are_strings_similar_lev(s, ai_s)
                for ai_s in ai_action["action_modified_sentences"]
            )
        ]

        # ---- Filter temporal order ----
        sentences_human = [
            s
            for s in action_human["sentences_temporal_order"]
            if not any(
                are_strings_similar_lev(s, ai_s)
                for ai_s in ai_action["action_modified_sentences"]
            )
        ]
        action_human["human_sentences_temporal_order"] = " ".join(sentences_human)

        return [ai_action, action_human]

    else:
        # ---- Purely human insert ----
        action_human = action.copy()
        action_human["level_1_action_type"] = "insert_text_human"

        sentences_human = [
            s
            for s in action_human["sentences_temporal_order"]
            if not any(are_strings_similar_lev(s, ai_s) for ai_s in ai_sentences)
        ]
        action_human["human_sentences_temporal_order"] = " ".join(sentences_human)

        return [action_human]

In [47]:
# Initialize an empty dictionary to store parsed actions
level_1_actions_per_session = {}

# Iterate through all sessions in the raw logs and parse actions
for session in tqdm(logs_by_session, desc="Parsing Level 1 Actions"):

    # Initialize the MergeActionsAnalyzer for each session
    actions_analyzer = action_parser.MergeActionsAnalyzer(
        last_action=None,
        raw_logs=logs_by_session[session]
    )

    # Parse the logs for the session into structured actions
    actions_lst, last_action = actions_analyzer.parse_actions_from_logs(
        all_logs=logs_by_session[session],
        last_action=None,
        DLT_CHAR_MAX_COUNT=9  # Optional: Specify tiny delete threshold here
    )

    # Store the parsed actions in the output dictionary
    level_1_actions_per_session[session] = actions_lst

# Add a new key to each action for classification and further analysis
for session_key, actions in level_1_actions_per_session.items():
    i = 0

    while i < len(actions):
        action = actions[i]
        if i>0 and action["action_type"] == "insert_text":
            split_actions = split_insert_text_by_delta(action, actions[i-1])
            if len(split_actions) == 1:
                actions[i] = split_actions[0]
                i += 1
            else:
                actions[i:i+1] = split_actions
                i += len(split_actions)
        else:
            action["level_1_action_type"] = action["action_type"]
            i += 1

Parsing Level 1 Actions: 100%|██████████| 11/11 [00:00<00:00, 31.70it/s]


In [48]:
level_1_actions_per_session.keys()

dict_keys(['legislation_corporate_1', 'legislation_corporate_2', 'legislation_corporate_3', 'legislation_corporate_4', 'legislation_corporate_5', 'legislation_antitrust_1', 'legislation_antitrust_2', 'legislation_antitrust_3', 'legislation_antitrust_4', 'legislation_antitrust_5', 'legislation_antitrust_6'])

In [49]:
[
    (action["level_1_action_type"], action.get("action_delta"))
    for action in level_1_actions_per_session["legislation_corporate_1"]
]

[('insert_text',
  ('INSERT', 'Corporate personhood has its pros and cons. ', 44, 7)),
 ('query_suggestion', ''),
 ('present_suggestion', ''),
 ('hover_over_text', ''),
 ('accept_suggestion', ''),
 ('present_suggestion', ''),
 ('insert_text_ai',
  ['INSERT',
   'Corporate personhood has its pros and cons. One advantage is that it allows corporations to enter contracts, sue or be sued, which promotes business activities and economic growth. However, this concept can lead to abuse of power, as some corporations might exploit these',
   381,
   58]),
 ('insert_text_human',
  ['INSERT',
   ". Let's talk about the advantages first. Corporate personhood allows corporations to have basic rights, which ",
   381,
   58]),
 ('query_suggestion', ''),
 ('present_suggestion', ''),
 ('hover_over_text', ''),
 ('cursor_operation', ''),
 ('reject_suggestion', ''),
 ('delete_text',
  ('DELETE', 'Corporate personhood has its pros and cons.', 43, 7)),
 ('cursor_operation', ''),
 ('insert_text_human',
  (

In [50]:
[
    action if action["action_type"] == "present_suggestion" else None
    for action in level_1_actions_per_session["legislation_corporate_1"]
]

[None,
 None,
 {'action_type': 'present_suggestion',
  'action_source': 'api',
  'action_logs': [{'eventName': 'suggestion-open',
    'eventSource': 'api',
    'eventTimestamp': 1750095856681,
    'textDelta': '',
    'cursorRange': '',
    'currentDoc': '',
    'currentCursor': 44,
    'currentSuggestions': [{'index': 0,
      'original': 'For instance, it allows companies the right to enter into contracts, sue or be sued, and own property as distinct legal entities.',
      'trimmed': 'For instance, it allows companies the right to enter into contracts, sue or be sued, and own property as distinct legal entities.',
      'probability': None,
      'source': 'GPT-4o'},
     {'index': 1,
      'original': 'However, understanding its implications can help us navigate the complex relationship between corporations, individuals, and society.',
      'trimmed': 'However, understanding its implications can help us navigate the complex relationship between corporations, individuals, and socie

In [51]:
[
    (action["level_1_action_type"], action.get("human_sentences_temporal_order"))
    for action in level_1_actions_per_session["legislation_corporate_1"]
    if "human_sentences_temporal_order" in action
]

[('insert_text_human',
  "Let's talk about the advantages first. Corporate personhood allows corporations to have basic rights, which"),
 ('insert_text_human',
  "Let's talk about the advantages first. Corporate personhood allows corporations to have basic rights, which is guaranteed in the constitution. People get rights - like vouching for yourself and others if there's unfair situation, which reflects a valid and fair justice system. So, corporate personhood gives companies responbiblity, which is empowering, and can be used to greater improve companies, the products they sell, and therefore society. The abuse of power"),
 ('insert_text_human',
  "Let's talk about the advantages first. Corporate personhood allows corporations to have basic rights, which is guaranteed in the constitution. People get rights - like vouching for yourself and others if there's unfair situation, which reflects a valid and fair justice system. So, corporate personhood gives companies responbiblity, which i

### Save Parsed Level 1 Actions to a JSON File in the Current Directory

This section saves the **level_1_actions_per_session** dictionary into a JSON file named `level_1_actions_per_session.json`. The file will be created in the same directory as this notebook for convenient access to the parsed output.

#### Key Fields Included in Each Entry:

- **action_type**: The type of the action (e.g., `insert_text`, `delete_text`).

- **action_source**: Specifies whether the action originated from the user or the API.  

- **action_logs**: A list of raw logs corresponding to the action.  

- **action_start_log_id**: The log ID where the action starts.  

- **action_start_time**: The timestamp of when the action begins.  

- **action_end_time**: The timestamp of when the action ends.  

- **writing_modified**: A boolean indicating whether the writing was modified during the action.  

- **action_delta**: The changes made to the text as a result of the action.  

- **action_end_writing**: The final state of the text after the action is completed.  

- **action_end_mask**: The final state of the mask after the action is completed.  

- **action_modified_sentences**: A list of sentences affected by the action.  

- **sentences_temporal_order**: The order of sentences after modifications.  

By saving the parsed data, you create a reusable file that can be used for additional analysis or as input for other tools.

In [52]:
# Define the output file path in the same directory as this notebook
output_file = os.path.join(script_dir, 'level_1_actions_per_session.json')

# Write the parsed actions dictionary to the JSON file
with open(output_file, 'w') as f:
    json.dump(level_1_actions_per_session, f, default=utils.custom_serializer)

### Level 2 Learning Parsing Based on Generated Level 1 Actions and Outcome Measures Documentation

#### Key Features:

The pipeline consists of two main steps:

1. **Level 2 Parsing**: Augments Level 1 actions with detailed Level 2 attributes.
    <br/>Attributes include:
- vibe_writing: prewriting opinions are opposed with genAI writing task
- minor_vibe_writing: prewriting opinions are potentially opposed with genAI writing task
- constructive_learning: applied prewriting learning to genAI writing task (over 3/4 of the writing was mentioned before in the prewriting)
- minor_constructive_learning: applied prewriting learning to some of genAI writing task (over 1/4 of the writing was mentioned before in the prewriting)
    
2. **Outcome Measures Computation**: Computes Stance Detection and Natural Language Inference

In [53]:
def parse_level_2_actions(level_1_actions_per_session):
    """
    Parses Level 2 actions based on Level 1 actions, but only for insert_text actions.

    Args:
        level_1_actions_per_session (dict): A dictionary where each session key maps to a list of level 1 actions.

    Returns:
        dict: The updated actions dictionary with Level 2 attributes added to each action.
    """

    for session_id, actions_lst in tqdm(
        level_1_actions_per_session.items(), desc="Parsing Level 2 Actions"
    ):
        past_ai_sentences = []
        past_arguments = []

        filename = utils.get_filename("prewriting_logs/", session_id)
        prewriting_content = ""

        with open(filename, "r", encoding="utf-8") as file:
            for line in file:
                data = json.loads(line)
                if "content" in data:
                    prewriting_content += data["content"]

        for idx, action in enumerate(actions_lst):
            try:
                if action["action_type"] == "present_suggestion" and action["action_logs"][0]["eventName"] == "suggestion-open":
                    for suggestion in action["action_logs"][0]["currentSuggestions"]:
                        past_ai_sentences.append(suggestion["trimmed"])

                # Only process insert_text_human actions, skip deletions
                if (
                    action.get("level_1_action_type") != "insert_text_human"
                    and action.get("level_1_action_type") != "insert_text_ai"
                ):
                    continue

                action["level_2_info"] = {}

                # Compute stance + nli with the pre-writing
                if (
                    action["level_1_action_type"] == "insert_text_human" 
                    and idx > 0
                    and "human_sentences_temporal_order" in action
                    and prewriting_content
                    and action["human_sentences_temporal_order"] != ""
                ):
                    writing_stance = level_2_learning_comparisons.get_stance_difference(
                        prewriting_content, action["human_sentences_temporal_order"]
                    )
                    writing_nli, nli_info_dct = level_2_learning_comparisons.get_NLI(
                        prewriting_content,
                        action["human_sentences_temporal_order"],
                    )
                    action["level_2_info"]["stance"] = writing_stance
                    action["level_2_info"]["nli"] = writing_nli
                    action["level_2_info"]["nli_info"] = nli_info_dct

                    if (
                        level_2_learning_comparisons.parse_level_2_vibe_writing(
                            writing_stance, writing_nli
                        )
                        == 1
                    ):
                        action["level_2_action_type"] = "vibe_writing"
                    elif (
                        level_2_learning_comparisons.parse_level_2_vibe_writing(
                            writing_stance, writing_nli
                        )
                        == 0.5
                    ):
                        action["level_2_action_type"] = "minor_vibe_writing"
                        
                if (
                    idx > 0
                    and "action_modified_sentences" in action
                    and action["action_modified_sentences"] != []
                ):
                    # constructive_learning_dict = (
                    #     level_2_learning_comparisons.get_constructive_learning(
                    #         prewriting_content, action["action_modified_sentences"]
                    #     )
                    # )
                    # parascores_dict = level_2_learning_comparisons.get_paraphrase_depth(
                    #     action["action_modified_sentences"],
                    #     level_2_learning_comparisons.get_relevant_background_info(
                    #         session_id, action["action_modified_sentences"],
                    #     )
                    #     | constructive_learning_dict,
                    # )
                    connectives_dict = (
                        level_2_learning_comparisons.get_discourse_connectives(
                            action["action_modified_sentences"]
                        )
                    )
                    arguments_dict = (
                        level_2_learning_comparisons.classify_argument_sentences(
                            " ".join(action["action_modified_sentences"])
                        )
                    )
                    new_arguments_dict = level_2_learning_comparisons.parse_argument_sentences(arguments_dict,past_arguments)
                    # print(past_arguments)
                    metacognitive_verbs_dict = level_2_learning_comparisons.detect_first_person_metacognitive_phrases(action["action_modified_sentences"])
                    hedges_dict = level_2_learning_comparisons.detect_hedging_labeled(action["action_modified_sentences"])
                    certainty_dict = level_2_learning_comparisons.detect_certainty(action["action_modified_sentences"])
                    external_examples_dict = level_2_learning_comparisons.get_external_example(
                        " ".join(action["action_modified_sentences"])
                    )
                    ai_parascores_dict = (
                        level_2_learning_comparisons.get_paraphrase_depth(
                            action["action_modified_sentences"],
                            level_2_learning_comparisons.get_relevant_background_info(
                                past_ai_sentences,
                                action["action_modified_sentences"],
                            )
                        )
                    )

                    # action["level_2_info"]["constructive_learning"] = constructive_learning_dict
                    # action["level_2_info"]["parascores"] = parascores_dict
                    action["level_2_info"]["connectives"] = connectives_dict
                    action["level_2_info"]["arguments"] = new_arguments_dict
                    action["level_2_info"]["metacognitive_verbs"] = metacognitive_verbs_dict
                    action["level_2_info"]["hedge_terms"] = hedges_dict
                    action["level_2_info"]["certainty_terms"] = certainty_dict
                    action["level_2_info"]["external_examples"] = external_examples_dict
                    action["level_2_info"]["ai_parascores"] = ai_parascores_dict

                    # if(len(action["action_delta"])>0 and action["action_delta"][1]!=""):
                    #     action["level_2_info"]["classify_text"] = level_2_learning_comparisons.classify_text(action["action_delta"][1])
                    #     classification = (
                    #         level_2_learning_comparisons.parse_classify_text(
                    #             action["level_2_info"]["classify_text"].items()
                    #         )
                    #     )
                    #     if(classification!=""):
                    #         action["text_classification"] = classification

                    # Determine the Level 2 action type
                    # constructive_learning = level_2_learning_comparisons.parse_level_2_constructive_learning(
                    #     constructive_learning_dict,
                    #     action["action_modified_sentences"],
                    # )
                    # paraphrasing = level_2_learning_comparisons.parse_level_2_paraphrase_depth(
                    #     parascores_dict
                    # )
                    if not "level_2_action_type" in action:
                        if len(new_arguments_dict)>1 or len(connectives_dict)>1:
                            action["level_2_action_type"] = "considered_mulitple_new_perspectives"
                        elif len(new_arguments_dict)==1 or len(connectives_dict)==1:
                            action["level_2_action_type"] = "considered_new_perspective"
                        elif len(metacognitive_verbs_dict) >= 1:
                            action["level_2_action_type"] = "metacognitive_writing"
                        elif len(hedges_dict) >= 1 and len(certainty_dict) >= 1:
                            action["level_2_action_type"] = "strong_cognitive_engagement"
                        elif len(hedges_dict) == 1 or len(certainty_dict) == 1:
                            action["level_2_action_type"] = "cognitive_engagement"  
                        # elif constructive_learning >= 0.75 and paraphrasing >= 0.8:
                        #     action["level_2_action_type"] = "constructive_learning_deep_paraphrasing"
                        # elif constructive_learning >= 0.75 and paraphrasing >= 0.2:
                        #     action["level_2_action_type"] = (
                        #         "constructive_learning_moderate_paraphrasing"
                        #     )
                        # elif constructive_learning >= 0.75 and paraphrasing < 0.2:
                        #     action["level_2_action_type"] = (
                        #         "constructive_learning_no_paraphrasing"
                        #     )
                        # elif constructive_learning >= 0.25 and paraphrasing >= 0.8:
                        #     action["level_2_action_type"] = "minor_constructive_learning_deep_paraphrasing"
                        # elif constructive_learning >= 0.25 and paraphrasing >= 0.2:
                        #     action["level_2_action_type"] = (
                        #         "minor_constructive_learning_moderate_paraphrasing"
                        #     )
                        # elif constructive_learning >= 0.25 and paraphrasing < 0.2:
                        #     action["level_2_action_type"] = (
                        #         "minor_constructive_learning_no_paraphrasing"
                        #     )
                        # elif constructive_learning < 0.25 and paraphrasing >= 0.2:
                        #     action["level_2_action_type"] = (
                        #         "no_constructive_learning_moderate_paraphrasing"
                        #     )
                        elif level_2_learning_comparisons.get_paraphrase_depth_suggestions(
                                ai_parascores_dict) >= 0.8:
                            action["level_2_action_type"] = "deep_paraphrasing_with_ai"
                        elif level_2_learning_comparisons.get_paraphrase_depth_suggestions(
                            ai_parascores_dict
                        ) >= 0.2:
                            action["level_2_action_type"] = "moderate_paraphrasing_with_ai"
                        elif level_2_learning_comparisons.parse_level_2_external_examples(
                                external_examples_dict
                            ) > 0.75:
                            action["level_2_action_type"] = "external_example"
                        # elif constructive_learning < 0.25 and paraphrasing < 0.2:
                        #     action["level_2_action_type"] = (
                        #         "no_constructive_learning_no_paraphrasing"
                        #     )

            # labeling which actions are major inserts
                if (
                    action["action_type"] == "insert_text"
                    and action["level_1_action_type"] == "insert_text_human"
                    and action["action_delta"] != ""
                    and action["action_delta"][-1] >= 10
                ):
                    action["level_2_insertion_type"] = "major_insert"

            except Exception as e:
                import traceback
                print(f"\n!! Error in session {session_id}, action index {idx} !!")
                traceback.print_exc()
                continue

    return level_1_actions_per_session


# Parse Level 2 actions
with open("level_1_actions_per_session.json", "r", encoding="utf-8") as f:
    level_1_actions_per_session = json.load(f)

level_2_actions_per_session = parse_level_2_actions(
    level_1_actions_per_session
)

Parsing Level 2 Actions:   0%|          | 0/11 [00:00<?, ?it/s]

'Corporate personhood has its pros and cons.' → {'label': 'NON-ARGUMENT', 'score': 0.954317569732666}
'One advantage is that it allows corporations to enter contracts, sue or be sued, which promotes business activities and economic growth.' → {'label': 'ARGUMENT', 'score': 0.7372101545333862}
'However, this concept can lead to abuse of power, as some corporations might exploit these' → {'label': 'ARGUMENT', 'score': 0.6203879714012146}
'Let's talk about the advantages first.' → {'label': 'NON-ARGUMENT', 'score': 0.9570932984352112}
'Corporate personhood allows corporations to have basic rights, which' → {'label': 'ARGUMENT', 'score': 0.9291141033172607}
'Corporate personhood allows corporations to have basic rights, which is guaranteed in the constitution.' → {'label': 'ARGUMENT', 'score': 0.9086834788322449}
'People get rights - like vouching for yourself and others if there's unfair situation, which reflects a valid and fair justice system.' → {'label': 'ARGUMENT', 'score': 0.7167578

Parsing Level 2 Actions:   9%|▉         | 1/11 [00:09<01:39,  9.96s/it]

'The only way to combat this is to somehow have a board of unbia' → {'label': 'NON-ARGUMENT', 'score': 0.7560921907424927}
'what are some arguments for corporate personhood Corporate personhood' → {'label': 'NON-ARGUMENT', 'score': 0.9775661826133728}
'what are some arguments for corporate personhood Corporate personhood -- the concept that corporations can be treated as singular entities separate from any body subsumed within them -- is, to me, an illogical opinion that has done much harm' → {'label': 'ARGUMENT', 'score': 0.5666748285293579}
'what are some arguments for corporate personhood Corporate personhood -- the concept that corporations can be treated as singular entities separate from any body subsumed within them -- is, to me, an illogical opinion that has doneharm to the United States and its democratic system.' → {'label': 'ARGUMENT', 'score': 0.6895865201950073}
'One of the most significant harms done by corporate personhood is in the Citizens United case, as mentioned in 

Parsing Level 2 Actions:  18%|█▊        | 2/11 [00:18<01:21,  9.06s/it]

...

'Antitrust regulations are laws that aim to prmote faair' → {'label': 'ARGUMENT', 'score': 0.5823298096656799}
'Antitrust regulations are laws that aim to pomote fair competition in consumer markets to ensure that' → {'label': 'ARGUMENT', 'score': 0.5581210851669312}
'Antitrust regulations are laws that aim to pomote fair competition in consumer markets toprevent the eirxploitation.' → {'label': 'ARGUMENT', 'score': 0.7942118048667908}
'They are most o' → {'label': 'NON-ARGUMENT', 'score': 0.9601403474807739}
'Antitrust regulations are laws that aim to pomote fair competition in consumer markets toprevent thploitation.Most familiarly, such regulations target the' → {'label': 'ARGUMENT', 'score': 0.6351922750473022}
'Antitrust regulations are laws that aim to pomote fair competition in consumer markets toprevent thploitation.Most familiarly, such regulationsoppose the formation of monopolies that would force a consumer to go' → {'label': 'ARGUMENT', 'score': 0.8112943172454834}
'Antitru

Parsing Level 2 Actions:  91%|█████████ | 10/11 [02:09<00:16, 16.74s/it]

'This creates a troubling dynamic where users may unknowingly sacrifice their privacy for convenience, illustrating the need for stricter regulations to safeguard personal data from exploitation in these zero-price markets.' → {'label': 'ARGUMENT', 'score': 0.9537720084190369}
'This means that companies may prioritize profit over user welfare, leading to exploitative practices.' → {'label': 'ARGUMENT', 'score': 0.976850152015686}
'This is already happening with companies, including Meta and Apple.' → {'label': 'NON-ARGUMENT', 'score': 0.8920936584472656}
'This creates a dangerous cycle where companies can prioritize collecting data without considering the long-term implications for users, potentially leading to harmful consequences in areas like identity theft or misinformation.' → {'label': 'ARGUMENT', 'score': 0.9029861092567444}
'Meta has been known for leading consumers down wormholes and polarizing individuals.' → {'label': 'NON-ARGUMENT', 'score': 0.98207688331604}
'People see a 

Parsing Level 2 Actions: 100%|██████████| 11/11 [02:12<00:00, 12.06s/it]

'To start addressing this issue, Antitrust laws surrounding unpaid services need modification, not only for consumer protection but also for fostering a fairer competitive landscape where businesses cannot thrive solely on exploitative practices aimed at maximizing profit, ultimately benefiting both consumers and the marketplace as a whole.' → {'label': 'ARGUMENT', 'score': 0.8049529790878296}


In [54]:
def parse_level_2_actions_suggestions(level_2_actions_per_session):
    """
    Parses Level 2 present_suggestion actions based on Level 2 actions.

    Args:
        level_2_actions_per_session (dict): A dictionary where each session key maps to a list of level 2 actions from insert_text actions

    Returns:
        dict: The updated actions dictionary with Level 2 attributes added to each action.
    """

    for session_id, actions_lst in tqdm(
        level_2_actions_per_session.items(), desc="Parsing Level 2 present_suggestion Actions"
    ):
        for idx, action in enumerate(actions_lst):
            try:

                # Only process insert_text_human actions, skip deletions
                if (
                    action.get("level_1_action_type") != "query_suggestion"
                    and action["action_logs"]
                    # and action["action_logs"][0]["eventName"] != "suggestion-get"
                ):
                    continue

                label = ""
                suggestion_open = action["action_logs"][0]
                if suggestion_open["currentCursor"] == len(action["action_start_writing"]) + 1:
                    label+="end_writing_"
                    stripped_writing = action["action_start_writing"].strip()
                    if stripped_writing[-1] in [".", "!", "?"] or action[
                        "action_start_writing"
                    ][-2:] =="\n":
                        label+="end_sentence"
                    else:
                        label+="middle_sentence"
                elif suggestion_open["currentCursor"] == 1:
                    label+="start_writing"
                else:
                    label += "middle_writing_"
                    writingBefore = action["action_start_writing"][:suggestion_open["currentCursor"]+1]
                    writingBefore = writingBefore.strip()
                    if writingBefore=="":
                        label = "start_writing"
                    elif (
                        writingBefore[-1]
                        in [".", "!", "?"]
                        or writingBefore[-2:] == "\n"
                    ):
                        label += "end_sentence"
                    else:
                        label += "middle_sentence"
                action["sentence_location"] = label

                sentences_before = utils.sent_tokenize(action["action_start_writing"])
                if sentences_before:
                    classify_text = level_2_learning_comparisons.classify_text(sentences_before[-1])
                    classification = (
                            level_2_learning_comparisons.parse_classify_text(
                                classify_text
                            )
                        )
                    if(classification!=""):
                        action["text_classification"] = classification
                

            except Exception as e:
                import traceback

                print(f"\n!! Error in session {session_id}, action index {idx} !!")
                traceback.print_exc()
                continue

    return level_2_actions_per_session

# Parse Level 2 present_suggestions actions
level_2_actions_per_session_suggestions = parse_level_2_actions_suggestions(level_2_actions_per_session)

Parsing Level 2 present_suggestion Actions: 100%|██████████| 11/11 [00:08<00:00,  1.30it/s]


In [55]:
# Compute outcome measures by iterating through each writing session after generating level 2 action types
for session_id, actions_lst in tqdm(
    level_2_actions_per_session_suggestions.items(), desc="Computing Outcome Measures"
):

    ##### Outcome measures applied to individual actions #####
    # Compute validity of major human text or suggestion insertions
    # outcome_measures.compute_validity(session_id, actions_lst)
    ##### End of individual action-based outcome measures #####

    ##### Outcome measures applied to the entire list of actions #####
    # Compute confidence of major human text 
    # outcome_measures.compute_confidence_linguistic(actions_lst)

    # outcome_measures.compute_conceptual_understanding(
    #     actions_lst)
    
    outcome_measures.compute_cognitive_levels(actions_lst)
    ##### End of list-wide outcome measures #####

Computing Outcome Measures: 100%|██████████| 11/11 [00:00<00:00, 614.49it/s]


In [56]:
[
    (
        action["action_type"],
        action.get("level_2_action_type"),
        action.get("action_delta"),
    )
    for action in level_2_actions_per_session["legislation_corporate_2"]
]

[('insert_text',
  None,
  ['INSERT', 'what are some arguments for corporate personhood', 48, 7]),
 ('query_suggestion', None, ''),
 ('present_suggestion', None, ''),
 ('hover_over_text', None, ''),
 ('insert_text', None, ['INSERT', '\n', 1, 0]),
 ('reject_suggestion', None, ''),
 ('insert_text',
  'considered_new_perspective',
  ['INSERT', '\nCorporate personhood ', 22, 2]),
 ('delete_text', None, ['DELETE', '\nCorporate personhood ', 22, 2]),
 ('insert_text',
  'considered_new_perspective',
  ['INSERT',
   'Corporate personhood -- the concept that corporations can be treated as singular entities separate from any body subsumed within them -- is, to me, an illogical opinion that has done much harm ',
   193,
   32]),
 ('delete_text', None, ['DELETE', ' much harm ', 11, 2]),
 ('insert_text',
  'considered_new_perspective',
  ['INSERT',
   'harm to the United States and its democratic system. \n\nOne of the most significant harms done by corporate personhood is in the Citizens United ca

In [57]:
counts = {}
for user in level_2_actions_per_session.keys():
    for action in level_2_actions_per_session[user]:
        if action.get("level_2_action_type"):
            if counts.get(action.get("level_2_action_type")):
                counts[action.get("level_2_action_type")] = (
                    counts[action.get("level_2_action_type")] + 1
                )
            else:
                counts[action.get("level_2_action_type")] = 1

counts

{'considered_new_perspective': 247,
 'considered_mulitple_new_perspectives': 32,
 'minor_vibe_writing': 13}

In [58]:
# Define the output file path
script_dir = os.getcwd()
output_file = os.path.join(script_dir, "level_2_actions_per_session_learning.json")

# Save the parsed actions to the JSON file
with open(output_file, "w") as f:
    json.dump(level_2_actions_per_session_suggestions, f, default=utils.custom_serializer)

# Confirm successful save
print(f"Level 2 actions successfully saved to: {output_file}")

Level 2 actions successfully saved to: /Users/chelseashe/Thought_Toolkit/level_2_actions_per_session_learning.json


### Level 3 Parsing Based on Generated Level 2 Actions

This function processes the **level_2_actions_per_session** dictionary and augments actions with Level 3 attributes, enabling more sophisticated analysis. Key enhancements include:

#### Key Features:

- **Tracking Accepted Suggestions**: Monitors the latest accepted AI suggestions to align user insertions with AI-generated content.  

- **Identifying Mindless Patterns**: Detects specific patterns for better insight into user behavior:  
  - **Mindless Echo**: Direct repetition of AI suggestions.  
  - **Mindless Edit**: Minor modifications to AI suggestions without meaningful changes.  

- **Detecting Topic Shifts**: Identifies the introduction of new ideas and maintains a running count of detected topic shifts.  

#### Threshold Options:

These thresholds can be configured at the top of the **level_3_comparisons.py** file:

- **MIN_INSERT_WORD_COUNT = 10**: Minimum word count required for echo detection.  

- **MAX_INSERT_WORD_COUNT = 3**: Maximum word count to classify as a mindless edit.  

- **MAX_SIMILARITY_ECHO = 0.93**: Maximum similarity threshold for mindless echo classification.  

- **MAX_SIMILARITY_MINDLESS_EDIT = 0.9**: Similarity threshold for detecting mindless edits.  

- **IDEA_ALIGNMENT_MAX_SIMILARITY = 0.6**: Maximum similarity threshold for idea alignment.  

- **IDEA_ALIGNMENT_MIN_WORD_COUNT = 10**: Minimum word count threshold for detecting new ideas.  

This step enhances the parsed data with advanced attributes, providing deeper insights into user interaction patterns and their alignment with AI suggestions.

In [59]:
def parse_level_3_actions(level_2_actions_per_session, similarity_fcn):
    """
    Processes level 2 actions and enhances them with level 3 attributes:
    - Tracks the latest accepted AI suggestions for alignment with user insertions.
    - Identifies patterns like mindless echoes or edits of AI suggestions.
    - Detects topic shifts and counts new ideas introduced by the user.

    Args:
        level_2_actions_per_session (dict): A dictionary where each session key maps to a list of level 2 actions.
        similarity_fcn (function): A similarity function (e.g., SpaCy) used to compare writing.

    Returns:
        dict: The updated actions dictionary with level 3 attributes added to each action.
    """
    for _, actions_lst in tqdm(
        level_2_actions_per_session.items(), desc="Parsing Level 3 Actions"
    ):
        latest_accepted_suggestion = ""  # Tracks the last accepted suggestion
        curr_idea_sentence_list = [""]  # Tracks current idea alignment sentences
        running_idea_count = 0  # Tracks the count of new ideas

        for action in actions_lst:
            try:
                action_parsed = False

                # Track the latest accepted suggestion
                if action.get("level_1_action_type") == "insert_text_ai":
                    latest_accepted_suggestion = action.get("action_delta")[1]
                    curr_idea_sentence_list = (
                        level_3_comparisons.get_idea_alignment_order_on_AI(
                            action, curr_idea_sentence_list, similarity_fcn
                        )
                    )
                    if len(curr_idea_sentence_list) == 1:  # New idea detected
                        running_idea_count += 1
                    action["topic_shift"] = running_idea_count

                # Process 'insert_text_human' when strings are not empty or whitespaces
                if (
                    action.get("level_1_action_type") == "insert_text_human"
                    and action.get("action_delta")
                    and action["action_delta"][1].strip()
                ):
                    # Check for major_insert_mindless_echo
                    echo_bool, echo_similarity, echo_details = (
                        level_3_comparisons.get_mindless_echo_after_AI(
                            action, latest_accepted_suggestion, similarity_fcn
                        )
                    )
                    if echo_similarity is not None:
                        action["level_3_info"] = {
                            **echo_details,
                            "similarity": echo_similarity,
                        }
                    if echo_bool:
                        action["level_3_action_type"] = "major_insert_mindless_echo"
                        action_parsed = True

                    # Check for minor_insert_mindless_edit if not already parsed
                    if not action_parsed:
                        edit_bool, edit_similarity, edit_details = (
                            level_3_comparisons.get_mindless_edit_of_AI(
                                action, latest_accepted_suggestion, similarity_fcn
                            )
                        )
                        if edit_similarity is not None:
                            action["level_3_info"] = {
                                **edit_details,
                                "similarity": edit_similarity,
                            }
                        if edit_bool:
                            action["level_3_action_type"] = "minor_insert_mindless_edit"
                            action_parsed = True

                    # Handle topic shifts for text insertions
                    if not action_parsed:
                        if (
                            action["action_delta"][-1]
                            >= level_3_comparisons.IDEA_ALIGNMENT_MIN_WORD_COUNT
                        ):
                            curr_idea_sentence_list = (
                                level_3_comparisons.get_idea_alignment_order_on_AI(
                                    action, curr_idea_sentence_list, similarity_fcn
                                )
                            )
                            if len(curr_idea_sentence_list) == 1:  # New idea detected
                                running_idea_count += 1
                            action["running_idea_count"] = running_idea_count
                            if running_idea_count > 0:
                                action["level_3_action_type"] = "topic_shifted"
                        else:
                            curr_idea_sentence_list, new_idea = (
                                level_3_comparisons.get_idea_alignment_order_on_minor_insert(
                                    action, curr_idea_sentence_list, similarity_fcn
                                )
                            )
                            if new_idea:
                                running_idea_count += 1
                            action["running_idea_count"] = running_idea_count
                            if running_idea_count > 0:
                                action["level_3_action_type"] = "topic_shifted"
            except Exception as e:
                print(f"There is an error in action '{action}': {e}")
                continue

    return level_2_actions_per_session


# Parse Level 3 actions and compute additional metrics
level_3_actions_per_session = parse_level_3_actions(
    level_2_actions_per_session_suggestions, similarity_fcn=utils.get_spacy_similarity
)

Parsing Level 3 Actions: 100%|██████████| 11/11 [00:36<00:00,  3.32s/it]


In [60]:
# Define the output file path
output_file = os.path.join(script_dir, "level_3_actions_per_session_learning.json")

# Save the parsed actions to the JSON file
with open(output_file, "w") as f:
    json.dump(level_3_actions_per_session, f, default=utils.custom_serializer)

# Confirm successful save
print(f"Level 3 actions successfully saved to: {output_file}")

Level 3 actions successfully saved to: /Users/chelseashe/Thought_Toolkit/level_3_actions_per_session_learning.json


### Level 4 Dividing each text insert by sentences

### Assigning Action Types with Default or Custom Priority Logic

This step determines and assigns the final `action_type` to actions in the dataset using either the default priority logic or a customizable priority list.

#### Key Functions:

1. **`determine_final_action_type_default`**
   - This function automatically determines the `action_type` for each action based on the following default priority logic:
     - **Level 2 Action Types**: "vibe_writing", "minor_vibe_writing"
     - **Level 3 Action Types**: "major_insert_mindless_echo", "major_insert_mindless_edit"
     - **Level 2 Action Types**: everything else
     - **Level 3 Action Types**: everything else
     - **Level 1 Action Types**: Default if neither Level 3 nor Level 2 types are present.

In [61]:
def determine_final_action_type_default(actions_dict):
    """
    Determines the final action type for each action in the actions dictionary.

    Default logic:
    - Level 3 action types have the highest priority and are considered first.
    - If no level 3 action type exists, the level 2 action type is used as the next priority.
    - If neither level 3 nor level 2 action types exist, the initial level 1 action type is retained.

    Args:
        actions_dict (dict): Dictionary where each session key maps to a list of actions.

    Returns:
        dict: The updated actions dictionary with 'action_type' set to the final action type.
    """
    for _, actions in actions_dict.items():
        for action in actions:
            if "level_3_action_type" in action and action["level_3_action_type"]:
                action["action_type"] = action["level_3_action_type"]
            elif "level_2_action_type" in action and action["level_2_action_type"]:
                action["action_type"] = action["level_2_action_type"]
            else:
                action["action_type"] = action["level_1_action_type"]
    return actions_dict

In [62]:
# Test with 'legislation_corporate_2'
test_session_dict = {
    "legislation_corporate_2": level_3_actions_per_session["legislation_corporate_2"]
}

updated_test_actions = determine_final_action_type_default(test_session_dict)

for action in updated_test_actions["legislation_corporate_2"]:
    print(action.get("action_type"))

insert_text
query_suggestion
present_suggestion
hover_over_text
insert_text_human
reject_suggestion
considered_new_perspective
delete_text
topic_shifted
delete_text
topic_shifted
query_suggestion
present_suggestion
hover_over_text
cursor_operation
reject_suggestion
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
cursor_operation
delete_text
cursor_operation
delete_text
cursor_operation
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted


In [63]:
def determine_final_action_type_customize(actions_dict, priority_list):
    """
    Sorts and assigns action types in the actions dictionary based on a custom priority list.

    Args:
        actions_dict (dict): Dictionary where each session key maps to a list of actions.
        priority_list (list): List of action types in descending order of priority.

    Returns:
        dict: The updated actions dictionary with 'action_type' assigned based on the custom priority list.
    """
    for _, actions in actions_dict.items():
        for action in actions:
            for priority_action in priority_list:
                if (
                    action.get("level_1_action_type") == priority_action
                    or action.get("level_2_action_type") == priority_action
                    or action.get("level_3_action_type") == priority_action
                ):
                    action["action_type"] = priority_action
                    break  # Assign the highest priority and stop checking
    return actions_dict

In [64]:
# Test with 'legislation_corporate_2'
test_session_dict = {
    "legislation_corporate_2": level_3_actions_per_session["legislation_corporate_2"]
}

custom_priority_list = [
    "vibe_writing",
    "minor_vibe_writing",
    "major_insert_mindless_echo",
    "major_insert_mindless_editing",
    "constructive_learning_deep_paraphrasing",
    "constructive_learning_moderate_paraphrasing",
    "minor_constructive_learning_deep_paraphrasing"
]
updated_test_actions = determine_final_action_type_customize(
    test_session_dict, custom_priority_list
)

for action in updated_test_actions["legislation_corporate_2"]:
    print(action.get("action_type"))

insert_text
query_suggestion
present_suggestion
hover_over_text
insert_text_human
reject_suggestion
considered_new_perspective
delete_text
topic_shifted
delete_text
topic_shifted
query_suggestion
present_suggestion
hover_over_text
cursor_operation
reject_suggestion
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
cursor_operation
delete_text
cursor_operation
delete_text
cursor_operation
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted


In [65]:
action_type_with_priority_per_session = determine_final_action_type_default(
    level_3_actions_per_session
)

In [66]:
# Define the output file path
output_file = os.path.join(
    script_dir, "action_type_with_priority_per_session_learning.json"
)

# Save the prioritized actions to the JSON file
with open(output_file, "w") as f:
    json.dump(action_type_with_priority_per_session, f, default=utils.custom_serializer)

# Confirm successful save
print(f"Prioritized action types saved to: {script_dir}")

Prioritized action types saved to: /Users/chelseashe/Thought_Toolkit
